In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
pip install fast_ml

You should consider upgrading via the 'c:\users\bilal\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [3]:
#call_data
df = pd.read_excel('rpi_volvo_de_test.xlsx')

In [4]:
df.head()

,VO_ANNONCE_ID,MARQUE,MODELE,CARBURANT,CARROSSERIE,ANNEE,REF_ID,REFERENCE_PROBABILITE,COTE_VO,PRIX_VENTE_TTC,...,KM,MODELE1,MODELE2,CARBURANT1,CARBURANT2,GENRE,METALIC_IND,SEGMENT_NAME,CROSSCOUNTRY,CARROSSERIE2
0,2,VOLVO,V60,DIESEL,BREAK,2016,110987,0.99,28973,21869.80,...,25989,V,60,DIESEL,DIESEL,VP,Y,MOYEN,N,BREAK
1,3,VOLVO,S60,DIESEL,BERLINE,2015,110872,0.99,32559,20907.12,...,2662,S,60,DIESEL,DIESEL,VP,Y,MOYEN,N,BERLINE
2,4,VOLVO,XC60,DIESEL,4X4 S.U.V,2016,111095,0.99,36742,30556.11,...,16951,XC,60,DIESEL,DIESEL,VP,Y,MINI_SUV,N,4X4 S.U.V
3,5,VOLVO,V60,DIESEL,BREAK,2016,110987,0.99,29036,21871.94,...,24770,V,60,DIESEL,DIESEL,VP,Y,MOYEN,N,BREAK
4,6,VOLVO,V70,DIESEL,BREAK,2016,65931,0.99,34458,27319.00,...,23955,V,70,DIESEL,DIESEL,VP,Y,MOYEN_GRAND,N,BREAK


In [5]:
df.shape

(27614, 21)

In [6]:
#Upon request we can reduce the frais de remise en état 'FRE' from the 'PRIX_VENTE_TTC', and so drop the column 'FRE'
df['PRIX_VENTE_TTC'] = df['PRIX_VENTE_TTC'] - df['FRE']

# Q1. What are the important features

There are many way to determine the important features. 
1. We look first if there are variables with a lot of missing data. In this case, it's better to drop this feature.
2. We can eliminate constant features functions by appealing the 'get_constant_features'. 
3. We look if there are repeated features. For example here, we shall see the difference between the features: 'Carburant', Carburant', 'Carburant2'.

  3.1. In case these features are distinct, then they are correlated and we shall drop two of them after looking at correlation matrix
4. We look finally the effect of each feature on the target variable. If the change of values in feature variables leads to significant change in the target variable, then this feature must be keeped. Otherwise, we can drop this feature   

In [7]:
# Explorotary data analysis 

# Total missing values for each feature
df.isnull().sum()

VO_ANNONCE_ID            0
MARQUE                   0
MODELE                   0
CARBURANT                0
CARROSSERIE              0
ANNEE                    0
REF_ID                   0
REFERENCE_PROBABILITE    0
COTE_VO                  0
PRIX_VENTE_TTC           0
FRE                      0
KM                       0
MODELE1                  0
MODELE2                  0
CARBURANT1               0
CARBURANT2               0
GENRE                    0
METALIC_IND              0
SEGMENT_NAME             0
CROSSCOUNTRY             0
CARROSSERIE2             0
dtype: int64

No missing values. No feature could be dropped at this stage.

In [8]:
# Explorotary data analysis 

#We can remark that the feature 'MODELE' is the combination of 'MODELE1' and 'MODELE2'. So we can drop the last two features.

df = df.drop(['MODELE1', 'MODELE2'], axis= 1)   # 'MODELE1' and 'MODELE2' included in 'MODELE'

In [9]:
#we look at the difference between the features 'CARBURANT', 'CARBURANT1' and 'CARBURANT2'
print(df['CARBURANT'].value_counts())
print(df['CARBURANT1'].value_counts())
print(df['CARBURANT2'].value_counts())

DIESEL                15767
ESSENCE                6812
DIESEL_ELECTRIQUE      2728
ESSENCE_ELECTRIQUE     2286
ELECTRIQUE               21
Name: CARBURANT, dtype: int64
DIESEL        15767
ESSENCE        6812
HYBRID         5014
ELECTRIQUE       21
Name: CARBURANT1, dtype: int64
DIESEL        18495
ESSENCE        9098
ELECTRIQUE       21
Name: CARBURANT2, dtype: int64


We can notice here that the difference between the above features 'CARBURANT' and 'CARBURANT2' is just the characteristic 'HYPRID' in 'CARBURANT1' is decomposed into 'DIESEL_ELECTRIQUE' and 'ESSENCE_ELECTRIQUE' in 'CARBURANT2'. So we can drop 'CARBURANT1' since this feature is included in 'CARBURANT'.

Moreover, we notice that if CARBURANT=DIESEL_ELECTRIQUE -> CARBURANT1=HYBRID and CARBURANT2=DIESEL ; (18495=15767+2728) if CARBURANT=ESSENCE_ELECTRIQUE -> CARBURANT1=HYBRID and CARBURANT2=ESSENCE (9098= 6812+2286)

So we can drop also 'CARBURANT2' since this feature is included in 'CARBURANT'.

In [10]:
df = df.drop(['CARBURANT1', 'CARBURANT2'], axis= 1) 

In [11]:
#same reasoning for the features 'CARROSSERIE' and 'CARROSSERIE2'
print(df['CARROSSERIE'].value_counts())
print(df['CARROSSERIE2'].value_counts())

4X4 S.U.V    17806
BREAK         6071
BERLINE       3737
Name: CARROSSERIE, dtype: int64
4X4 S.U.V        15955
BREAK             6071
BERLINE           3737
CROSS-COUNTRY     1851
Name: CARROSSERIE2, dtype: int64


In [12]:
df = df.drop(['CARROSSERIE'], axis= 1)   # 'CAROSSERIE' is included in 'CARROSSERIE2'

In [13]:
print(df['CROSSCOUNTRY'].value_counts())

N    25763
Y     1851
Name: CROSSCOUNTRY, dtype: int64


In [14]:
# We can also remark that this feature is also included in 'CARROSSERIE2' (look at the number 1851)
df = df.drop(['CROSSCOUNTRY'], axis= 1) 

In [15]:
#Use the get_constant_features functions to get all the constant features.

from fast_ml.utilities import display_all
from fast_ml.feature_selection import get_constant_features

constant_features = get_constant_features(df)
constant_features.head(10)

,Desc,Var,Value,Perc
0,Constant,MARQUE,VOLVO,100.0
1,Constant,GENRE,VP,100.0


In [16]:
# drop 'MARQUE', 'GENRE' from get_constant_features functions

print(len(df["GENRE"].unique())) # for verification
print(len(df["MARQUE"].unique())) # for verification

# drop 'VO_ANNONCE_ID' as all values in this features are numerical and distinct

print(len(df["VO_ANNONCE_ID"].unique())) # for verification

# we expect that the feature 'REF_ID' contains distincts numerical values, however it's not the case
  #print(len(df["REF_ID"].unique()))= 594, and so we can't drop this feature.

1
1
27614


In [17]:
columns_to_keep = ["REF_ID", "REFERENCE_PROBABILITE", "METALIC_IND", "SEGMENT_NAME",
                   "CARROSSERIE2", "CARBURANT", "MODELE", "ANNEE", "COTE_VO",
                   "FRE", "KM", "PRIX_VENTE_TTC"]
data = df[columns_to_keep]

* formal observation:

Up to now, we have dropped severals features based on observation (feature included in an another, constant features, different numerical values in each features, etc). 


In [18]:
# decompose columns into categorial columns and numerical columns
categorical_cols = ["REF_ID","METALIC_IND","SEGMENT_NAME", "CARROSSERIE2", "CARBURANT", "MODELE"]
numerical_cols   = ["REFERENCE_PROBABILITE", "ANNEE", "COTE_VO","FRE", "KM", "PRIX_VENTE_TTC"]

# we can look at the correlation matrix between the numerical features
display(print(data.corr()))

                         REF_ID  REFERENCE_PROBABILITE     ANNEE   COTE_VO  \
REF_ID                 1.000000               0.039403  0.200142  0.145712   
REFERENCE_PROBABILITE  0.039403               1.000000 -0.365119 -0.219806   
ANNEE                  0.200142              -0.365119  1.000000  0.186307   
COTE_VO                0.145712              -0.219806  0.186307  1.000000   
FRE                   -0.066390               0.060954 -0.175324  0.042158   
KM                    -0.086651               0.076160 -0.206383 -0.024571   
PRIX_VENTE_TTC         0.164448              -0.262633  0.327125  0.918086   

                            FRE        KM  PRIX_VENTE_TTC  
REF_ID                -0.066390 -0.086651        0.164448  
REFERENCE_PROBABILITE  0.060954  0.076160       -0.262633  
ANNEE                 -0.175324 -0.206383        0.327125  
COTE_VO                0.042158 -0.024571        0.918086  
FRE                    1.000000  0.954194        0.018143  
KM             

None

Conclusion: it's clear that the feature 'PRIX_VENTE_TTC' is strongly related to the feature 'COTE_VO', and so this feature must be between the important features. Between the rest, we notice that the feature 'PRIX_VENTE_TTC' is also correlated to the feature 'ANNEE'.

Remark: the above observations were for numerical features. For categorial features I tried to search online in something similar and the result of my research is to transform my categorial data into numerical data. However, this will increase the number of features, and as a consequence this will reduce the performance of our algorithm. It may exist another way to look for important features between the categorial features that I don't know....

# GradientBoosting Algorithm

At the first stage, I thought about using "MultivariateRegression" to predict the value of 'PRIX_VENTE_TTC' in term of the other features. However, as I need an alogrithm that shall be able to determine the important features, then I choosed to use 'GradientBoosting' algorithm since:

"""A benefit of using gradient boosting is that after the boosted trees are constructed, it is relatively straightforward to retrieve importance scores for each attribute."""

In [19]:
#first step: convert categorial data to numerical data using 

from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
data_to_encode = data[categorical_cols]
encoded_data = enc.fit_transform(data_to_encode)

cats = [ ele for sublist in enc.categories_ for ele in sublist] 
categorical_data = pd.DataFrame(encoded_data.toarray(), columns = cats)
numerical_data = data[numerical_cols]
new_data = pd.concat([numerical_data,categorical_data], axis=1)
print(new_data.shape)

X = new_data.loc[:, new_data.columns != "PRIX_VENTE_TTC"]
y = new_data["PRIX_VENTE_TTC"]

(27614, 627)


In [20]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score, cross_val_predict
reg = GradientBoostingRegressor()



# validation
results  = cross_val_score(reg, X, y, cv= 5, scoring = "r2")  
print ("the average of r2 across folds is {avg} with std of {sig}".format(avg = np.mean(results),
                                                                            sig = np.std(results)))   

c:\users\bilal\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int64', 'str']. An error will be raised in 1.2.
  warnings.warn(
c:\users\bilal\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int64', 'str']. An error will be raised in 1.2.
  warnings.warn(
c:\users\bilal\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int64', 'str']. An error will be raised in 1.2.
  warnings.warn(
c:\users\bilal\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Go

the average of r2 across folds is 0.8305017110805677 with std of 0.0318671373345452


In [21]:
# predict

predictions  = cross_val_predict(reg, X, y, cv= 3)  
print(predictions)

c:\users\bilal\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int64', 'str']. An error will be raised in 1.2.
  warnings.warn(
c:\users\bilal\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int64', 'str']. An error will be raised in 1.2.
  warnings.warn(
c:\users\bilal\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int64', 'str']. An error will be raised in 1.2.
  warnings.warn(
c:\users\bilal\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Go

[25937.65346201 28073.12755714 34134.97632218 ... 64529.72486686
 52592.28190024 40811.64549955]


In [22]:
# features importance
reg.fit(X,y)
feature_importances = reg.feature_importances_

df_feature_importances = pd.DataFrame(feature_importances.reshape(1,-1), columns = X.columns)
df_feature_importances.T.sort_values(ascending=False, by=0).head(5)

c:\users\bilal\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int64', 'str']. An error will be raised in 1.2.
  warnings.warn(


,0
COTE_VO,0.917735
ANNEE,0.030099
4X4 S.U.V,0.013188
ESSENCE_ELECTRIQUE,0.010138
XC90,0.004193


Again here, we remark that the most important feature is 'COTE_VO'

# Conclusion

In [23]:
# the efficiency of our algorithm; compare the errors


# error_1 between 'PRIX_VENTE_TTC' and 'COTE-VO'

diff1 = new_data['COTE_VO'].values - y.values
sq1 = np.square(diff1)
mse1 = sq1.mean()
print(mse1)

# error_2 between 'Predictions' and 'COTE_VO'

diff2 = new_data['COTE_VO'].values - predictions
sq2 = np.square(diff2)
mse2 = sq2.mean()
print(mse2)



69182588.73115434
45389295.54407308


It's clear with our new values of 'PRIX_VENTE_TTC', the difference between 'PRIX_VENTE_TTC' and 'COTE_VO' is less than above and that's the objectif of this project!